In [ ]:
# 분석용도로 Spark processing  (user행동분석 / system분석) user행동분석을 가정한다고 했을 때
# 저장용도로 Spark processing  (제거 후 총 용량이 몇 퍼센트 감소했는지)



'''
필수 과제 : web access log 데이터 중 활용 가능한 데이터를 찾아서 적재 기준을 정의한다.
선택 과제 : 대용량 데이터 핸들링을 위한 플랫폼 구현
'''

'''
- 로그 분석 목적 :
방문객들이 웹 사이트를 방문하면서 남긴 흔적들을 분석함으로써 인터넷 비즈니스에 전략적으로 활용
보안이라는 기준에서 정상적인 접속시도와 불법적인 공격시도를 파악
사이트의 정보 설계를 경제적으로 할 수 있게 도와준다.

- 로그 항목 선별 및 정제 :
대용량의 데이터를 목적에 맞게 분석하고 활용하기 위해 어떠한 기준으로 데이터를 줄일 것 인가?

분석할 항목들을 설정한 후 분석에 필요한 항목들만 로그에 남겨 지도록해야한다.
아무런 설정없이 로그데이터를 받게 되면 로그 데이터의 크기는 매우 커지게 된다.
이렇게 되면 쌓이는 데이터의 크기만으로 분석을 할 수 없는 상황이 도래할 수 있다.

따라서, 로그데이터를 분석 가능한 형태로 바꿔주는 작업이 필요하다.

'''

'''
다우기술의 web access log format 

Apache NCSA extended combined log format + option(go-agent, processing time)

액세스 로그(access log)
- 사용자가 보낸 요청과 관련된 정보와 서버에서 어떤 응답을 보냈는지 등에 관련된 정보를 제공

리퍼러 로그(referer log)
- 웹사이트에 방문하는 사용자가 이전에 어디 위치에 있었는지 정보를 제공

에이전트 로그(Agent log)
- 웹 브라우저의 이름, 버전 등의 정보를 제공하는 로그

Option log
- go-agent : Go-Agent가 자신에 대해 알리는 식별정보
- T : 서버가 처리하는데 걸린시간(성능측정요소)

위와 같은 포맷을 토대로 13개의 컬럼생성
'''

'''
- 적재 기준
# 행 제거 기준 


'''


# 열 제거 기준
# 제거했을 때 총 용량이 몇 퍼센트 감소했는지?
# 1주일 단위로 몇퍼센트가 감소했는지?

In [1]:
## UserAgentParser
class UserAgentParser: 
    from user_agents import parse #// VERSION: 2.2.0
    from collections import namedtuple
    UA3Layers = namedtuple(typename='UA3Layers', field_names=['str_browser_name', 'str_operating_system_name', 'str_hardware_type_name'])
    
    class HardwareType:
        BOT   = 'Bot'
        TABLET   = 'Tablet'
        PHONE    = 'Phone'
        COMPUTER = 'Computer'
        OTHER    = 'Other'
        
    def get_3layers(self, str_user_agent):
        user_agent = self.__class__.parse(str_user_agent)
        return self.__class__.UA3Layers(self.get_browser_name(user_agent), self.get_operating_system_name(user_agent), self.get_hardware_type(user_agent))
 
    def get_browser_name(self, user_agent):
        """
        Return: ['Chrome', 'Firefox', 'Opera', 'IE', 'Edge', 'Safari', ...]
        변환로직:
            'IE Mobile' >>> 'IE'
            'Mobile Safari' >>> 'Safari'
        """
        str_browser_name = user_agent.browser.family
        str_browser_name = str_browser_name.replace('Mobile', '').strip()
        return str_browser_name
    
    def get_operating_system_name(self, user_agent):
        """
        Return: ['Windows','Linux','Mac OS X','iOS','Android','OpenBSD','BlackBerry OS','Chrome OS',...]
        """
        str_operating_system_name = user_agent.os.family
        return str_operating_system_name
    
    def get_hardware_type(self, user_agent):
        if user_agent.is_bot:
            str_hardware_type_name = self.__class__.HardwareType.BOT
        elif user_agent.is_tablet:
            str_hardware_type_name = self.__class__.HardwareType.TABLET
        elif user_agent.is_mobile:
            str_hardware_type_name = self.__class__.HardwareType.PHONE
        elif user_agent.is_pc:
            str_hardware_type_name = self.__class__.HardwareType.COMPUTER
        else:
            str_hardware_type_name = self.__class__.HardwareType.OTHER
        return str_hardware_type_name
    
#     def get_is_bot(self, user_agent):
#         return user_agent.is_bot

def extract_browser(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[0]

def extract_os(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[1]

def extract_device(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[2]

def trans_time(t):
    return str(pd.to_datetime(t,format='%d/%b/%Y:%H:%M:%S'))

In [2]:
import pandas as pd
from user_agents import parse


columns=["IP","identifier","email","time","request","url","protocol","status_code",\
         "bytesize","referer","user-agent","go-agent","processing time"]

path = '/home/jmyeong/tarfile/temp/access_2022-01-06.txt'

df = pd.read_csv(path ,sep=' ',header=None, names=columns)

df.reset_index(inplace = True)
del df["time"]
df.columns = columns
df["time"] = df["time"].apply(lambda x : x.replace("[",""))

for i in ('07','08','09','10'):

    path = f"/home/jmyeong/tarfile/temp/access_2022-01-{i}.txt"

    rawData = pd.read_csv(path ,sep=' ',header=None, names=columns)

    rawData.reset_index(inplace = True)
    del rawData["time"]
    rawData.columns = columns
    rawData["time"] = rawData["time"].apply(lambda x : x.replace("[",""))
    df = pd.concat([df, rawData])
    
#rawData.to_csv(path ,mode='w', header=False)

/home/jmyeong/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/jmyeong/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df1 = df[:1000000]

In [ ]:
# 1. user-agent에 있는 정보를 가지고 3개의 컬럼생성 (dataframe 생성)
df1["browser"] = df1["user-agent"].apply(lambda x : extract_browser(x))
df1["os"] = df1["user-agent"].apply(lambda x : extract_os(x))
df1["device"] =df1["user-agent"].apply(lambda x : extract_device(x))
#df1.drop("user-agent",axis=1, inplace=True)  # 따라서 identifier 컬럼제거

/tmp/ipykernel_23195/4063800640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["browser"] = df1["user-agent"].apply(lambda x : extract_browser(x))


In [14]:
# 오류처리
df1 = df1[(df1['time']!='-') | (df1["protocol"] == 'HTTP/1.0') | (df1["protocol"]=='HTTP/1.1')]

In [ ]:
# --------------------------------------------------------------------
## 행 기준

# device
df1 = df1[df1['device'] != 'Server'] # bot제거
df1 = df1[df1['device'] != 'Other'] # 개발과정에서 발생한 로그(크롤링, curl,Zabbix, okhttp 등)

# browser
select_browser = ["Chrome",'Other', 'Whale', 'Safari UI/WKWebView', 'IE', 'Edge', 'Chrome WebView', 'Safari', 'Firefox', 'Outlook', 'Samsung Internet', 'Opera','Chrome iOS','Firefox iOS','Outlook-iOS']
df1 = df1[df1['browser'].isin(select_browser)] # 개발과정에서 발생한 로그 제외

# os
df1 = df1[df1["os"] != 'Other'] # 알 수 없는 os, 개발과정에서 발생한 로그 제외

# 2000000개 중 1397750 약 30%감소
# 3000000개 중 2131143  약 30%감소

In [5]:
df1[df1['device'] != 'Bot']

,IP,identifier,email,time,request,url,protocol,status_code,bytesize,referer,go-agent,processing time,browser,os,device
0,39.7.46.217,-,-,06/Jan/2022:19:57:45,PROPFIND,/.well-known/carddav,HTTP/1.0,401,-,-,-,0.329,Other,iOS,Other
1,39.7.46.217,-,-,06/Jan/2022:19:57:45,PROPFIND,/.well-known/carddav,HTTP/1.0,401,-,-,-,0.348,Other,iOS,Other
2,-,-,-,06/Jan/2022:19:57:46,GET,/internal/healthcheck,HTTP/1.1,400,138,-,-,0.899,Java,Other,Server
3,-,-,-,06/Jan/2022:19:57:46,GET,/internal/healthcheck,HTTP/1.1,400,138,-,-,0.887,Java,Other,Server
4,211.217.131.173,-,-,06/Jan/2022:19:57:47,GET,/,HTTP/1.0,302,-,-,-,1.576,Edge,Windows,Computer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899971,172.21.25.99,-,wldnjs8236@daou.co.kr,07/Jan/2022:15:38:26,POST,/api/approval/integration/hr/remainhdcnt,HTTP/1.0,200,204,https://portal.daou.co.kr/app/approval/documen...,-,0.061,Chrome,Windows,Computer
899972,172.21.25.65,-,mogg@daou.co.kr,07/Jan/2022:15:38:27,GET,/api/chat/buddies/4646?size=20&userSeq=2033,HTTP/1.0,200,1398,-,GO-PC,0.131,Electron,Windows,Computer
899973,58.124.46.197,-,hhhy12345@daou.co.kr,07/Jan/2022:15:38:27,POST,/api/mail/message/flag,HTTP/1.0,200,110,https://portal.daou.co.kr/app/mail/?state=1,GO-iPhone,3.079,Safari UI/WKWebView,iOS,Phone
899974,172.21.25.89,-,bizppurio@daou.co.kr,07/Jan/2022:15:38:27,GET,/api/mail/folder/all,HTTP/1.0,200,1613,https://portal.daou.co.kr/app/mail/home,-,0.063,IE,Windows,Computer


In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'matplotlib'

In [6]:
# 실제사용자가 아닌 bot
## 시각화 

bar_df = df1["browser"].value_counts().reset_index()

bar_df

plt.title("Bar Plot", fontsize=15)
df1["browser"].value_counts().plot(kind='bar')
# plt.bar(bar_df["index"], df1["browser"],color='k', alpha=.3)
plt.ylabel("사람 수", fontsize=12)
plt.xlabel("생존여부", fontsize=12)
plt.show()

,IP,identifier,email,time,request,url,protocol,status_code,bytesize,referer,user-agent,go-agent,processing time
0,39.7.46.217,-,-,06/Jan/2022:19:57:45,PROPFIND,/.well-known/carddav,HTTP/1.0,401,-,-,iOS/14.8 (18H17) dataaccessd/1.0,-,0.329
1,39.7.46.217,-,-,06/Jan/2022:19:57:45,PROPFIND,/.well-known/carddav,HTTP/1.0,401,-,-,iOS/14.8 (18H17) dataaccessd/1.0,-,0.348
2,-,-,-,06/Jan/2022:19:57:46,GET,/internal/healthcheck,HTTP/1.1,400,138,-,Java/1.8.0_162,-,0.899
3,-,-,-,06/Jan/2022:19:57:46,GET,/internal/healthcheck,HTTP/1.1,400,138,-,Java/1.8.0_162,-,0.887
4,211.217.131.173,-,-,06/Jan/2022:19:57:47,GET,/,HTTP/1.0,302,-,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,-,1.576
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400727,221.153.15.125,-,swpark@daou.co.kr,10/Jan/2022:10:43:01,GET,/app/works/applet/6635/doc/1770966/navigate,HTTP/1.0,200,998,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,-,0.052
400728,211.252.84.65,-,-,10/Jan/2022:10:43:01,GET,/openapi/meeting/token/validation?token=eyJ0eX...,HTTP/1.0,200,141,-,-,-,0.011
400729,211.252.84.65,-,-,10/Jan/2022:10:43:01,GET,/openapi/meeting/token/validation?token=eyJ0eX...,HTTP/1.0,200,141,-,-,-,0.004
400730,172.21.22.131,-,john@daou.co.kr,10/Jan/2022:10:43:01,GET,/api/meeting/room,HTTP/1.0,200,374,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,GO-PC,0.685


In [1]:
# -----------------------------------------------------------------------
## 컬럼 기준

# identifier 컬럼 제거 
print(len(df1)) # 1397750
print(len(df1[df1['identifier'] != '-'])) # 0
df1.drop("identifier",axis=1, inplace=True)  # 따라서 identifier 컬럼제거

# go-agent 컬럼 제거
'''
'BROWSER'  = 'Android', 'iOS'
'GO-Android' = Android
'GO-PC' = Windows
'GO-iPhone' = iOS
'undefined' = 'Android', 'Mac OS X', 'Windows', 'iOS' 

 user-agent의 (browser, os, device)를 파악함으로써 go-agent의 정보를 대체할 수 있다.
'''
df1.drop("go-agent",axis=1, inplace=True)  # 따라서 go-agent 컬럼제거

# protocol 컬럼 제거 (status_code를 통하여 http/1.0 protocol를 유추가능)

print(len(df1["protocol"])) # 1397750
print(set(df1["protocol"])) # {'HTTP/1.0'}
df1.drop("protocol",axis=1, inplace=True)  

# bot(user-agent) device == 'Server'
'''
'Java/1.8.0_111',
 'Java/1.8.0_162',
 'Java/1.8.0_172',
 'Java/1.8.0_92',
 'Java/11.0.9',
 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/601.2.4 (KHTML, like Gecko) Version/9.0.1 Safari/601.2.4 facebookexternalhit/1.1 Facebot Twitterbot/1.0',
 'Mozilla/5.0 (Windows NT 10.0; ZumBot/1.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534+ (KHTML, like Gecko) BingPreview/1.0b',
 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
 'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
 'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)',
 'Mozilla/5.0 (compatible;PetalBot;+https://webmaster.petalsearch.com/site/petalbot)',
 'Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/96.0.4664.93 Safari/537.36',
 'TelegramBot (like TwitterBot)',
 'facebookexternalhit/1.1; kakaotalk-scrap/1.0; +https://devtalk.kakao.com/t/scrap/33984'
'''

# 개발시 발생 로그, 시스템 로그(user-agent) device == 'Other'
'''
"'Mozilla/4.0",
 '-',
 'Apache-HttpClient/4.5.10 (Java/1.8.0_152)',
 'Apache-HttpClient/4.5.12 (Java/1.8.0_151)',
 'Apache-HttpClient/4.5.2 (Java/1.8.0_162)',
 'CalDavSynchronizer/4.1',
 'DEXT5E61d78131',
 'DEXT5E61d7a225',
 'DEXT5E61d7ab26',
 'DEXT5E61d7bf9c',
 'DEXT5E61d7db1b',
 'DaouOffice/10.3.0.20 (com.daou.GroupOfficeInhouse; build:20211118; iOS 15.1.1) Alamofire/5.4.3',
 'Expanse indexes the network perimeters of our customers. If you have any questions or concerns, please reach out to: scaninfo@expanseinc.com',
 'GO-PC/9.5.1.1',
 'Go http package',
 'Jakarta Commons-HttpClient/3.0',
 'Microsoft Office Excel 2014',
 'Microsoft Office Protocol Discovery',
 'Microsoft Office/16.0 (Microsoft Outlook 16.0.14326; Pro), Mozilla/4.0 (compatible; ms-office; MSOffice rmj)',
 'Mozilla/4.0 (compatible; ms-office; MSOffice 16)',
 'Mozilla/4.0 (compatible; ms-office; MSOffice rmj)',
 'Mozilla/5.0',
 'Mozilla/5.0 (compatible; CensysInspect/1.1; +https://about.censys.io/)',
 'Mozilla/5.0 (compatible; NetcraftSurveyAgent/1.0; +info@netcraft.com)',
 'Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)',
 'Mozilla/5.0 zgrab/0.x',
 'NateOn/7.0.10.0 (5254)',
 'NaverMailApp/2.3.9',
 'SCMGUARD',
 'Superhuman',
 'Zabbix',
 'curl/7.29.0',
 'curl/7.41.0',
 'electron-builder',
 'iOS/13.3 (17C54) dataaccessd/1.0',
 'iOS/14.4 (18D52) dataaccessd/1.0',
 'iOS/14.6 (18F72) dataaccessd/1.0',
 'iOS/14.7 (18G68) dataaccessd/1.0',
 'iOS/14.7 (18G69) dataaccessd/1.0',
 'iOS/14.7.1 (18G82) dataaccessd/1.0',
 'iOS/14.8 (18H17) dataaccessd/1.0',
 'iOS/14.8 (18H17) remindd/1.0',
 'iOS/14.8.1 (18H107) dataaccessd/1.0',
 'iOS/14.8.1 (18H107) remindd/1.0',
 'iOS/15.0.1 (19A348) dataaccessd/1.0',
 'iOS/15.0.2 (19A404) dataaccessd/1.0',
 'iOS/15.0.2 (19A404) remindd/1.0',
 'iOS/15.1 (19B74) dataaccessd/1.0',
 'iOS/15.1 (19B74) remindd/1.0',
 'iOS/15.1.1 (19B81) dataaccessd/1.0',
 'iOS/15.2 (19C56) dataaccessd/1.0',
 'iOS/15.2 (19C56) remindd/1.0',
 'iOS/15.2 (19C57) remindd/1.0',
 'okhttp/3.12.0',
 'okhttp/3.14.8',
 'okhttp/3.8.0',
 'python-requests/2.17.3',
 'python-requests/2.22.0',
 'python-requests/2.23.0',
 'python-requests/2.25.1',
 'python-requests/2.26.0',
 'python-requests/2.6.0 CPython/2.7.5 Linux/3.10.0-1062.9.1.el7.x86_64',
 'python-requests/2.6.0 CPython/2.7.5 Linux/3.10.0-1160.21.1.el7.x86_64',
 'python-requests/2.6.0 CPython/2.7.5 Linux/3.10.0-514.el7.x86_64',
 'python-requests/2.6.0 CPython/2.7.5 Linux/3.10.0-693.el7.x86_64',
 'python-requests/2.6.0 CPython/2.7.5 Linux/3.10.0-957.1.3.el7.x86_64'}
'''



ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.